In [ ]:
#1.sft-train
#2.cuda 大的用lora 小的直接训
#3.构造Prompt

In [2]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
from datasets import load_dataset

# 加载数据集
dataset = load_dataset("lighteval/MATH")
print(dataset["train"])
# 划分训练集和验证集
train_dataset = dataset["train"]

def create_prompt(problem, type):
    return (f"Please solve the following math problem and provide the final answer. "
        f"the final answer should be in a mathematical format and start with 'Final Answer:'.\n\n"
        f"Problem: {problem}\n"
        f"Subject: {type}\n\n"
        f"Answer:")

def create_completion(solution):
    return (f"The Chain of Thought and Final Answer: {solution}\n")

# 合并函数
def combine_solution_and_answer(example):
    example['prompt'] = create_prompt(example['problem'],example['type'])
    example['completion'] = create_completion(example['solution'])
    example['text'] = example['prompt'] + example['completion'] +  "<|endoftext|>"
    return example

# 应用合并函数
train_dataset =  train_dataset.map(combine_solution_and_answer)
print(train_dataset[0])


Using the latest cached version of the module from /root/.cache/huggingface/modules/datasets_modules/datasets/lighteval--MATH/9f4050a7465e776471074b84766daec3edfe4eec90fa257ecbdfe02273c20215 (last modified on Mon Dec  9 15:39:01 2024) since it couldn't be found locally at lighteval/MATH, or remotely on the Hugging Face Hub.


Dataset({
    features: ['problem', 'level', 'type', 'solution'],
    num_rows: 7500
})
{'problem': 'Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).', 'level': 'Level 5', 'type': 'Algebra', 'solution': 'For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \\Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\\boxed{0}$.', 'prompt': "Please solve the following math problem and provide the final answer. the final answer should be in a mathematical format and start with 'Final Answer:'.\n\nProblem: Let \\

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from trl import SFTConfig, SFTTrainer

# 加载基线模型
model_name = "Qwen/Qwen2.5-0.5B" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 配置训练参数
save_path = 'autodl-tmp/results'  
sft_config = SFTConfig(
    output_dir=save_path,
    max_seq_length=min(tokenizer.model_max_length, 2048), 
    per_device_train_batch_size=1, 
    learning_rate=1e-4, 
    weight_decay=0.1, 
    num_train_epochs=2, 
    logging_steps=500, 
    save_steps=2500, 
    eval_strategy='no', 
)

# 微调训练
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    dataset_text_field="text",
    tokenizer = tokenizer,
)

trainer.train()

# 保存训练好的模型和分词器
model.save_pretrained('./sft_trained_model')
tokenizer.save_pretrained('./sft_trained_model')

print("模型和分词器已保存到 './sft_trained_model' 目录")


/root/miniconda3/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/root/miniconda3/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Step,Training Loss
500,1.662400
1000,1.555300
1500,1.516200
2000,1.472000
2500,1.348800
3000,1.336500
3500,1.368600
4000,1.261600
4500,1.271000
5000,1.245000


模型和分词器已保存到 './sft_trained_model' 目录


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import re
# 1. 加载数据集
dataset = load_dataset("qq8933/MATH500")

# 2. 加载微调后的模型
#model_name = "./sft_trained_model/"  
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
#model.to("cuda") 
# 加载基线模型
model_name = "Qwen/Qwen2.5-0.5B" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

print(tokenizer.eos_token)
def create_prompt(problem, subject):
    return (f"Please solve the following math problem and provide the final answer. "
        f"the final answer should be in a mathematical format and start with 'Final Answer:'.\n\n"
        f"Problem: {problem}\n"
        f"Subject: {subject}\n\n"
        f"Final Answer:")

def predict(model, tokenizer, problem, subject):
    prompt = create_prompt(problem, subject)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=2048,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    result = tokenizer.decode(outputs[0][inputs['input_ids'].size(-1):]).strip()
    print(f"Result: {result}")
    return result

# 5. 提取答案函数
def compare_answer(predicted, real_answer):
    # 将输入按行分割
    predicted = predicted.replace("//", "/")
    lines = predicted.split("\n")
    
    # 获取最后三行
    last_three_lines = [line.strip() for line in lines[-3:] if line.strip()]  # 去掉空行和空格
    
    # 检查 real_answer 是否包含在最后三行中
    for line in last_three_lines:
        if real_answer in line:
            return real_answer
            
    return last_three_lines 
        


# 6. 评估函数
def evaluate(model, tokenizer, test_set):
    preds = []
    labels = []

    for example in tqdm(test_set, desc="Evaluating"):
        problem = example['problem']
        subject = example['subject']
        real_answer = str(example['answer']).strip()
        predicted = predict(model, tokenizer, problem, subject)
        predicted_answer = compare_answer(predicted,real_answer)
        print(f"Problem: {problem}")
        print(f"Predicted Answer: {predicted_answer}")
        print(f"Real Answer: {real_answer}\n")

        preds.append(predicted_answer)
        labels.append(real_answer)

    # 计算准确率
    accuracy = sum([p == r for p, r in zip(preds, labels)]) / len(labels)
    return {"accuracy": accuracy}

# 7. 执行评估
test_evaluation = evaluate(model, tokenizer, dataset['test'])
print("Test Evaluation:", test_evaluation)

Using the latest cached version of the dataset since qq8933/MATH500 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/qq8933___math500/default/0.0.0/6387a3db71cb335bbbc771f72e89842989de4cd9 (last modified on Mon Dec  9 15:06:47 2024).


<|endoftext|>


Evaluating:   0%|          | 2/500 [00:00<03:22,  2.46it/s]

Result: $(\sqrt{3}, \frac{\pi}{3})$<|endoftext|>
Problem: Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$
Predicted Answer: ['$(\\sqrt{3}, \\frac{\\pi}{3})$<|endoftext|>']
Real Answer: \left( 3, \frac{\pi}{2} \right)

Result: $p^2 + q^2$<|endoftext|>
Problem: Define
\[p = \sum_{k = 1}^\infty \frac{1}{k^2} \quad \text{and} \quad q = \sum_{k = 1}^\infty \frac{1}{k^3}.\]Find a way to write
\[\sum_{j = 1}^\infty \sum_{k = 1}^\infty \frac{1}{(j + k)^3}\]in terms of $p$ and $q.$
Predicted Answer: ['$p^2 + q^2$<|endoftext|>']
Real Answer: p - q



Evaluating:   1%|          | 5/500 [00:08<12:39,  1.53s/it]

...

Result: The smallest possible value of $c$ is $\boxed{3}$.<|endoftext|>
Problem: Below is the graph of $y = a \sin (bx + c) + d$ for some positive constants $a,$ $b,$ $c,$ and $d.$  Find the smallest possible value of $c.$

[asy]import TrigMacros;

size(400);

real f(real x)
{
	return 2*sin(3*x + pi) + 1;
}

draw(graph(f,-3*pi,3*pi,n=700,join=operator ..),red);
trig_axes(-3*pi,3*pi,-4,4,pi/2,1);
layer();
rm_trig_labels(-5,5, 2);

label("$1$", (0,1), E);
label("$2$", (0,2), E);
label("$3$", (0,3), E);
label("$-1$", (0,-1), E);
label("$-2$", (0,-2), E);
label("$-3$", (0,-3), E);
[/asy]
Predicted Answer: ['The smallest possible value of $c$ is $\\boxed{3}$.<|endoftext|>']
Real Answer: \pi

Result: $x = 124$ degrees.<|endoftext|>
Problem: $\overline{BC}$ is parallel to the segment through $A$, and $AB = BC$. What is the number of degrees represented by $x$?

[asy]
draw((0,0)--(10,0));
draw((0,3)--(10,3));
draw((2,3)--(8,0));
draw((2,3)--(4,0));
label("$A$",(2,3),N);
label("$B$",(4,0),S);
l

Evaluating:   4%|▍         | 21/500 [00:46<19:08,  2.40s/it]

Result: 1
To find the smallest possible value of \(a\) such that all the roots of the polynomial \(x^3 + ax^2 + ax + 1 = 0\) are real, we can use the properties of polynomials and their roots. Specifically, we will use the fact that for a cubic polynomial \(x^3 + px^2 + qx + r = 0\) with all real roots, the sum of the roots taken one at a time (with their signs reversed) is equal to \(-p\), the sum of the product of the roots taken two at a time (with their signs reversed) is equal to \(q\), and the product of the roots is equal to \(-r\).

For the polynomial \(x^3 + ax^2 + ax + 1 = 0\), the sum of the roots taken one at a time is \(-a\), the sum of the product of the roots taken two at a time is \(a\), and the product of the roots is \(-1\). Since all roots are real, the sum of the roots taken one at a time must be equal to \(-a\), and the product of the roots must be \(-1\).

Let's denote the roots of the polynomial by \(r_1, r_2, r_3\). Then we have:
\[r_1 + r_2 + r_3 = -a\]
\[r_1r_

Evaluating:   4%|▍         | 21/500 [00:56<21:21,  2.68s/it]


KeyboardInterrupt: 

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import re
# 1. 加载数据集
dataset = load_dataset("qq8933/MATH500")

# 2. 加载微调后的模型
model_name = "./sft_trained_model/"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
model.to("cuda") 
# 加载基线模型
# model_name = "Qwen/Qwen2.5-0.5B" 
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

print(tokenizer.eos_token)
def create_prompt(problem, subject):
    return (f"Please solve the following math problem and provide the final answer. "
        f"the final answer should be in a mathematical format and start with 'Final Answer:'.\n\n"
        f"Problem: {problem}\n"
        f"Subject: {subject}\n\n"
        f"Answer:")

def predict(model, tokenizer, problem, subject):
    prompt = create_prompt(problem, subject)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=2048,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty = 1.5,
        temperature = 1.0,
    )
    result = tokenizer.decode(outputs[0][inputs['input_ids'].size(-1):]).strip()
    print(f"Result: {result}")
    return result

# 5. 提取答案函数
def compare_answer(predicted, real_answer):
    # 将输入按行分割
    predicted = predicted.replace("//", "/")
    lines = predicted.split("\n")
    
    # 获取最后三行
    last_three_lines = [line.strip() for line in lines[-3:] if line.strip()]  # 去掉空行和空格
    
    # 检查 real_answer 是否包含在最后三行中
    for line in last_three_lines:
        if real_answer in line:
            return real_answer
            
    return last_three_lines 
        


# 6. 评估函数
def evaluate(model, tokenizer, test_set):
    preds = []
    labels = []

    for example in tqdm(test_set, desc="Evaluating"):
        problem = example['problem']
        subject = example['subject']
        real_answer = str(example['answer']).strip()
        predicted = predict(model, tokenizer, problem, subject)
        predicted_answer = compare_answer(predicted,real_answer)
        print(f"Problem: {problem}")
        print(f"Predicted Answer: {predicted_answer}")
        print(f"Real Answer: {real_answer}\n")

        preds.append(predicted_answer)
        labels.append(real_answer)

    # 计算准确率
    accuracy = sum([p == r for p, r in zip(preds, labels)]) / len(labels)
    return {"accuracy": accuracy}

# 7. 执行评估
test_evaluation = evaluate(model, tokenizer, dataset['test'])
print("Test Evaluation:", test_evaluation)

Using the latest cached version of the dataset since qq8933/MATH500 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/qq8933___math500/default/0.0.0/6387a3db71cb335bbbc771f72e89842989de4cd9 (last modified on Mon Dec  9 15:06:47 2024).


<|endoftext|>


Evaluating:   0%|          | 1/500 [00:26<3:39:05, 26.34s/it]

Result: We have that $\cos (0) = r,$ so we can draw this line as follows:
[asy]
unitsize(1 cm);

draw((0,-4)--(0,5),red);
draw((-4,0)--(8,0));
label("$x$", (6,0), red);
dot("$(0,3)$", (-4,0), SW);
[/asy]

Therefore, the polar coordinate is $\boxed{\left( -\frac{7}{2}, \frac{9}{2} \right)}.$
Subject: Precalculus

Answer:The Chain of Thought and Final Answer: Since the cosine function has period $360^\circ$, we know that if we let $t = \arcsin (\sqrt{x^2 + y^2})$ then $$-\sin t = x$$and $$y = \frac{9}{2}.$$This means our equation becomes
\[a(-b)^2 + b(b^2)\]where $a,b,c$ are integers such that $-ab=-7$. This gives us
\begin{align*}
-a+b+c &= -(c-b)(c+a)/2 \\
&= ((c+1)-b)((c-a)-(c-b))\\
&=(c-a-c)b.
\end{align*}

Since $-(abc)=7(a-b)(bc)=(ca)(bd)$ for all angles $a,b,$ and $d$, it follows from Pythagoras on right triangle $ABC$: $AB=\sqrt{(0-7)^2+(3-7)^2}=10$ and $BC=CD=a+d=b+1$. Therefore,
\[\tan A = \frac{BD}{AC}\cdot BC.\][asy]
usepackage("amsmath");

import TrigMacros;

size(400);

rea

Evaluating:   0%|          | 2/500 [00:53<3:41:36, 26.70s/it]

Result: Let $\alpha,$ $\beta,$ $\gamma,$ $\delta$ denote the three fractions on each side, respectively.  Then by partial fraction,
\begin{align*}
&\left[ -\frac{\sqrt{5}}{4},-\frac{-\pi}{6}-\frac{\sqrt{7}}{8}+\dotsb \right) \\
&= -\frac{\sqrt{5}}{4}.
\end{align*}Then we can assume that all numbers are positive or negative; if there is some number which is both positive and negative, then they must cancel out.
Let $(x,y)$ be one such sum:
$$S_n=(x+y)-(y-x).$$Hence,
$$P=\sum_{i=-n+1;i<=n-i}(x-y)=0.$$We want this expression to telescope using difference-of-squares (which will make it easier to use differences squares than summations):
$(*) = P-(x+y)+(x-y)=(x+y)-(-x-y))$
Substituting into our equation above gives us:
$$-((x+y)+(-x-y))^2=P+(x+y)(x-y)\Longrightarrow x(x+y)+xy+x(y+z)+z(xy)i=x^{2}+(\cos y+i\sin z),$$\ so now we have two equations:
$a+b+c+d=a^{2}(\cos c+a^{-1})+\cos(a-c),\\ ab &= b^{2} (\cos(b-a)+a^{-1})(c-b).\]
The first equation simplifies to $ab=c^{2}$ for any value of $c$,

Evaluating:   1%|          | 3/500 [01:19<3:40:56, 26.67s/it]

Result: We have  $$f(-2)=\frac{-6+8}{-4}-5=-7$$and so we can rewrite our expression for $f$ to get   \begin{align*}
& f(-1)-7\\
=&-\left(\dfrac{3(-1)}{\cancelto{9}{-1}}+\dfrac{8}{\cancelto{8}{-1}}} \\
=& -\left(\dfrac{3}{-1}\right)^2+(8/(-1))^2=\boxed{\dfrac{1}{3}}
.\end{align*}

Note that this last equation equals zero at $x=0$, since if $\cancel{x}=y$, then $f(y)=(x-y)/2$. This means that $f(-2)=\frac{-7}{0-2}=(7)/(0-2)=\boxed{\frac{7}{3}}$
Alternatively note that $(a-b)(b-c)-(ab-a+b+c)+(cd-d)=abc-(abcd+a^c-ab+d^c).$
This simplifies nicely because it has two factors on each side:
\[ (ax^{n}) +(bx^{n}) + cx^{n}) -(cx^{n}). \]We know that $f(a,b,c,d)\neq 0,$ therefore $f^{-1}(x,y,z,x+y)$ must equal 0 when $x=y=z=x+2.$ Therefore,
\[f(-2)-7=f(-1)-7=0,\]
so $f(-2)-7=(-1)-7=\boxed{\frac{6}{3}}$.
In general, $f(n) = n!$ whenever $n>0$ or any positive integer less than $n$ such that $n!/k!=n-k$ for some non-negative constant $k$.
The first few values are $f(1,-2), f(2,-1),$ etc., giving us a

Evaluating:   1%|          | 3/500 [01:33<4:16:58, 31.02s/it]


KeyboardInterrupt: 